## Import Libraries

In [14]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torch.utils.data import DataLoader, TensorDataset, random_split

## Data Preparation

In [15]:
df=pd.read_excel("AirQualityUCI.xlsx")
df.dropna(inplace=True)
df['month'] = df['Date'].dt.month
df

,Date,Time,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH,month
0,2004-03-10,18:00:00,2.6,1360.00,150,11.881723,1045.50,166.0,1056.25,113.0,1692.00,1267.50,13.600,48.875001,0.757754,3
1,2004-03-10,19:00:00,2.0,1292.25,112,9.397165,954.75,103.0,1173.75,92.0,1558.75,972.25,13.300,47.700000,0.725487,3
2,2004-03-10,20:00:00,2.2,1402.00,88,8.997817,939.25,131.0,1140.00,114.0,1554.50,1074.00,11.900,53.975000,0.750239,3
3,2004-03-10,21:00:00,2.2,1375.50,80,9.228796,948.25,172.0,1092.00,122.0,1583.75,1203.25,11.000,60.000000,0.786713,3
4,2004-03-10,22:00:00,1.6,1272.25,51,6.518224,835.50,131.0,1205.00,116.0,1490.00,1110.00,11.150,59.575001,0.788794,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9352,2005-04-04,10:00:00,3.1,1314.25,-200,13.529605,1101.25,471.7,538.50,189.8,1374.25,1728.50,21.850,29.250000,0.756824,4
9353,2005-04-04,11:00:00,2.4,1162.50,-200,11.355157,1027.00,353.3,603.75,179.2,1263.50,1269.00,24.325,23.725000,0.711864,4
9354,2005-04-04,12:00:00,2.4,1142.00,-200,12.374538,1062.50,293.0,603.25,174.7,1240.75,1092.00,26.900,18.350000,0.640649,4
9355,2005-04-04,13:00:00,2.1,1002.50,-200,9.547187,960.50,234.5,701.50,155.7,1041.00,769.75,28.325,13.550000,0.513866,4


In [16]:
for i in df.columns:
    if df[i].dtypes=="object":
        df[i] = df[i].astype('category')
        df[i] =df[i].cat.codes

In [17]:
targets = df[["PT08.S1(CO)"]].to_numpy()
inputs = df.drop(["PT08.S1(CO)","Date"], axis=1).to_numpy()  

In [18]:
inputs = torch.from_numpy(inputs)
inputs = inputs.float()
targets = torch.from_numpy(targets)
targets = targets.float()

In [19]:
inputs_mean = torch.mean(inputs, axis=0)
targets_mean = torch.mean(targets, axis=0)
inputs = inputs / inputs_mean  
targets = targets / targets_mean 

In [20]:
dataset = TensorDataset(inputs, targets)
a=len(dataset)
b=round(0.9*a)
c=a-b
train_dataset, val_dataset = torch.utils.data.dataset.random_split(dataset,[b,c])

In [21]:
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader=DataLoader(val_dataset, batch_size=64, shuffle=True)

## Define Two Models

In [24]:
model = nn.Sequential(nn.Linear(14, 8),
                      nn.Sigmoid(),
                       
                      nn.Linear(8, 4),
                      nn.Softmax(),
                       
                       
                      nn.Linear(4, 2),
                      nn.ReLU(),

                      
                      nn.Linear(2, 1),
                      nn.ReLU())

model_with_optimizer = nn.Sequential(nn.Linear(14, 8),
                      nn.Dropout(p=0.2),
                       
                      nn.Linear(8, 4),
                      nn.Softmax(),
                       
                       
                      nn.Linear(4, 2),
                      nn.ReLU(),

                      
                      nn.Linear(2, 1),
                      nn.ReLU())

In [37]:
# Utility function to train the model
def fit(num_epochs, model, criterion, optimizer, train_loader):
    running_loss = 0.0
    loss_values = []
    # Repeat for given number of epochs
    for epoch in range(num_epochs):
        # Train with batches of data
        for xb,yb in train_loader:
            # 1. Generate predictions
            pred = model(xb)
            # 2. Calculate loss
            loss = criterion(pred, yb)
            # 3. Compute gradients
            loss.backward()
            # 4. Update parameters using gradients
            optimizer.step()
            # 5. Reset the gradients to zero
            optimizer.zero_grad()
            running_loss += loss.item()
            loss_values.append(running_loss)
        # Print the progress
        if (epoch+1) % 10 == 0:
            print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

## Fit Model

In [41]:
criterion = F.mse_loss
optimizer = torch.optim.SGD(model.parameters(), lr=1e-4)
fit(200, model, criterion, optimizer, train_loader)

Epoch [10/200], Loss: 0.0853
Epoch [20/200], Loss: 0.0942
Epoch [30/200], Loss: 0.1299
Epoch [40/200], Loss: 0.1397
Epoch [50/200], Loss: 0.0616
Epoch [60/200], Loss: 0.1290
Epoch [70/200], Loss: 0.0729
Epoch [80/200], Loss: 0.1865
Epoch [90/200], Loss: 0.0280
Epoch [100/200], Loss: 0.1065
Epoch [110/200], Loss: 0.0930
Epoch [120/200], Loss: 0.0834
Epoch [130/200], Loss: 0.0727
Epoch [140/200], Loss: 0.0765
Epoch [150/200], Loss: 0.1179
Epoch [160/200], Loss: 0.0466
Epoch [170/200], Loss: 0.1073
Epoch [180/200], Loss: 0.1427
Epoch [190/200], Loss: 0.1070
Epoch [200/200], Loss: 0.1110


## Fit Model with Regularization

In [45]:
criterion = F.mse_loss
optimizer = torch.optim.SGD(model.parameters(),weight_decay=2, lr=1e-4)
fit(200, model, criterion, optimizer, train_loader)

Epoch [10/200], Loss: 0.3390
Epoch [20/200], Loss: 0.3127
Epoch [30/200], Loss: 0.3559
Epoch [40/200], Loss: 0.3837
Epoch [50/200], Loss: 0.3278
Epoch [60/200], Loss: 0.3681
Epoch [70/200], Loss: 0.3288
Epoch [80/200], Loss: 0.3825
Epoch [90/200], Loss: 0.3494
Epoch [100/200], Loss: 0.3926
Epoch [110/200], Loss: 0.3515
Epoch [120/200], Loss: 0.3226
Epoch [130/200], Loss: 0.3483
Epoch [140/200], Loss: 0.3645
Epoch [150/200], Loss: 0.4234
Epoch [160/200], Loss: 0.3995
Epoch [170/200], Loss: 0.2890
Epoch [180/200], Loss: 0.3298
Epoch [190/200], Loss: 0.4176
Epoch [200/200], Loss: 0.3367


## Fit Model with Regularization and Dropout

In [46]:
criterion = F.mse_loss
optimizer = torch.optim.SGD(model_with_optimizer.parameters(),weight_decay=2, lr=1e-4)
fit(200, model_with_optimizer, criterion, optimizer, train_loader)

Epoch [10/200], Loss: 0.3928
Epoch [20/200], Loss: 0.3157
Epoch [30/200], Loss: 0.3965
Epoch [40/200], Loss: 0.2967
Epoch [50/200], Loss: 0.3840
Epoch [60/200], Loss: 0.3792
Epoch [70/200], Loss: 0.3637
Epoch [80/200], Loss: 0.4221
Epoch [90/200], Loss: 0.4251
Epoch [100/200], Loss: 0.3441
Epoch [110/200], Loss: 0.3679
Epoch [120/200], Loss: 0.3417
Epoch [130/200], Loss: 0.3285
Epoch [140/200], Loss: 0.3364
Epoch [150/200], Loss: 0.3095
Epoch [160/200], Loss: 0.3588
Epoch [170/200], Loss: 0.3364
Epoch [180/200], Loss: 0.4139
Epoch [190/200], Loss: 0.3435
Epoch [200/200], Loss: 0.3079
